
---

### Happiness data


<div class="alert alert-block alert-info">
    
* Begin by importing the data on happiness. Sadly, the data is only available in several separate data sets. Import the data on the years 2015 through 2019 and merge them. You are going to need to identify and fix problems with incompatible column names.
</div>

In [1]:
# Import all important packages:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np

In [2]:
# Set up the working directory:
path = "C:/Users/gross/OneDrive/Dokumente/Grossepieper_Nils_Assignment02"
os.chdir(path)

# Import the data sets for the years 2015 up to 2019:
happy_2015 = pd.read_csv("happy_data/happy_2015.csv", na_values = ["", "NA"])
happy_2016 = pd.read_csv("happy_data/happy_2016.csv", na_values = ["", "NA"])
happy_2017 = pd.read_csv("happy_data/happy_2017.csv", na_values = ["", "NA"])
happy_2018 = pd.read_csv("happy_data/happy_2018.csv", na_values = ["", "NA"])
happy_2019 = pd.read_csv("happy_data/happy_2019.csv", na_values = ["", "NA"])

# Add a year variable for every data set:
happy_2015["Year"] = [2015] * happy_2015.shape[0]
happy_2016["Year"] = [2016] * happy_2016.shape[0]
happy_2017["Year"] = [2017] * happy_2017.shape[0]
happy_2018["Year"] = [2018] * happy_2018.shape[0]
happy_2019["Year"] = [2019] * happy_2019.shape[0]

# Set subsets containig only the important variables:
happy_2015 = happy_2015[["Year", "Country", "Happiness Score"]]
happy_2016 = happy_2016[["Year", "Country", "Happiness Score"]]
happy_2017 = happy_2017[["Year", "Country", "Happiness.Score"]]
happy_2018 = happy_2018[["Year", "Country or region", "Score"]]
happy_2019 = happy_2019[["Year", "Country or region", "Score"]]

# Change all variables names to the naming convetion of happy_2017:
happy_2015 = happy_2015.rename(columns = {
    "Happiness Score": "Happiness.Score"
    })
happy_2016 = happy_2016.rename(columns = {
    "Happiness Score": "Happiness.Score"
    })
happy_2018 = happy_2018.rename(columns = {
    "Score": "Happiness.Score",
    "Country or region": "Country"
    })
happy_2019 = happy_2019.rename(columns = {
    "Score": "Happiness.Score",
    "Country or region": "Country" 
    })

# Now we can merge the data:
happy = pd.concat([happy_2015, happy_2016, happy_2017, happy_2018,
                   happy_2019])
happy.head(10)

,Year,Country,Happiness.Score
0,2015,Switzerland,7.587
1,2015,Iceland,7.561
2,2015,Denmark,7.527
3,2015,Norway,7.522
4,2015,Canada,7.427
5,2015,Finland,7.406
6,2015,Netherlands,7.378
7,2015,Sweden,7.364
8,2015,New Zealand,7.286
9,2015,Australia,7.284


---

# Inflation data

Data on inflation is available from World Bank. 


<div class="alert alert-block alert-info">
    
* Download the most current version of the data from [their web site](https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG). 
* Import the data calling it `inflation` and turn it into a tidy format.

</div>

In [3]:
# Import the data inflation:
inflation = pd.read_csv("API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_4701153/API_FP.CPI.TOTL.ZG_DS2_en_csv_v2_4701153.csv",
                        na_values = ["", "NA"], skiprows = 4)

# Set subsets containig only the important variables:
inflation = inflation[["Country Name", "2015", "2016", "2017", "2018", "2019"]]

# Rename the variable Country Name:
inflation = inflation.rename(columns = {"Country Name": "Country"})

# Transform the data set into tidy data:
inflation_tidy = inflation.melt(id_vars = ["Country"], var_name = "Year",
                                value_name = "Inflation")
inflation_tidy.head(10)

,Country,Year,Inflation
0,Aruba,2015,0.474764
1,Africa Eastern and Southern,2015,5.250171
2,Afghanistan,2015,-0.661709
3,Africa Western and Central,2015,2.130268
4,Angola,2015,9.150372
5,Albania,2015,3.501208
6,Andorra,2015,NaN
7,Arab World,2015,1.814077
8,United Arab Emirates,2015,4.069966
9,Argentina,2015,NaN


---

# Merging the data

<div class="alert alert-block alert-info">

* Merge the two data frames `happy` and `inflation` by country and year. Only keep those observations for which the key exists in both tables.     
    
</div>

In [5]:
# Look at the types of the variables and bring it in the right format:
happy.dtypes
inflation_tidy.dtypes
inflation_tidy["Year"] = inflation_tidy["Year"].astype("int")

# Do an inner merge for the two data sets happy and inflation_tidy:
happy_inflation = pd.merge(happy, inflation_tidy, how = "inner",
                           on = ["Year", "Country"])
happy_inflation.head(10)

,Year,Country,Happiness.Score,Inflation
0,2015,Switzerland,7.587,-1.143909
1,2015,Iceland,7.561,1.633056
2,2015,Denmark,7.527,0.452034
3,2015,Norway,7.522,2.171137
4,2015,Canada,7.427,1.125241
5,2015,Finland,7.406,-0.207929
6,2015,Netherlands,7.378,0.600248
7,2015,Sweden,7.364,-0.046785
8,2015,New Zealand,7.286,0.292705
9,2015,Australia,7.284,1.508367


<div class="alert alert-block alert-info">

* Which table contains data about more countries than the other? How many of these countries are dropped by merging the data sets?
    
</div>

In [7]:
# First I check how many unique countries are contained in each of the two 
# data sets "happy" and "inflation_tidy":

unique_countries_happy = happy["Country"].unique()
print("Number of unique countries in happy:", str(len(unique_countries_happy)))   
unique_countries_inflation = inflation_tidy["Country"].unique()
print("Number of unique countries in inflation:", str(len(unique_countries_inflation)))

# You can see that the data set "inflation_tidy" contains more unique countries.
# I can not expect that the inner merge of these two has more than 170 countries, 
# since "happy" only contains 170 unique countries.

# When I look at which unique countires are contained in both data sets, then
# I can see how many unique countries should end up in the inner merge of the
# two data sets.

unique_countries_merge = happy_inflation["Country"].unique()
print("Number of unique countries in happy_inflation:", str(len(unique_countries_merge))) 

# 98 countries have been dropped due to the merging process.

Number of unique countries in happy: 170
Number of unique countries in inflation: 266
Number of unique countries in happy_inflation: 142


---

# Fixed effects regression

At this point we want to analyze possible effects of inflation on the happiness in a country. 

<div class="alert alert-block alert-info">

* Estimate a simple linear model, regressing the inflation rate on the happiness score. What do you find? Interpret the coefficients and discuss the statistical significance of the findings. Using a strong example from the data, show the economic significance of the findings by comparing inflation rates of two countries of your choice.

</div>    